# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# !pip install gmaps
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

# !jupyter nbextention list
# !jupyter nbextention enable --py gmaps
# from ipywidgets.embed import embed_minimal_html

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [36]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [37]:
vacation_cities = pd.read_csv('city_weather_info.csv')
vacation_cities.head()

,Unnamed: 0,city name,country,date,lon,lat,max temp,humidity,cloudiness,wind speed
0,0,Bredasdorp,ZA,1589995437,20.04,-34.53,55.40,82,0,5.82
1,1,Jardim,BR,1589995437,-56.14,-21.48,86.81,49,40,10.25
2,2,Mataura,NZ,1589995437,168.86,-46.19,36.99,82,94,3.58
3,3,Puerto Ayora,EC,1589995438,-90.35,-0.74,82.40,65,40,19.46
4,4,Nizhniy Baskunchak,RU,1589995438,46.83,48.22,56.01,88,100,12.84


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [38]:
gmaps.configure(api_key=g_key)

In [39]:
locations = vacation_cities[['lat','lon']]
humidity = vacation_cities['humidity']
fig = gmaps.figure(map_type = 'HYBRID', center = (0,0), zoom_level = 3)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [40]:
Figure(layout=FigureLayout(height='420px'))

NameError: name 'Figure' is not defined

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
narrowed_city_df = vacation_cities.loc[(vacation_cities["max temp"] < 80) & (vacation_cities["max temp"] > 70) \
                                    & (vacation_cities["wind speed"] < 10) \
                                    & (vacation_cities["cloudiness"] == 0)].dropna()
narrowed_city_df

,Unnamed: 0,city name,country,date,lon,lat,max temp,humidity,cloudiness,wind speed
41,41,Tigzirt,DZ,1589995440,4.12,36.89,73.09,49,0,4.50
105,105,Malanje,AO,1589995445,16.34,-9.54,70.21,55,0,3.27
137,137,Cidreira,BR,1589995447,-50.21,-30.18,72.19,59,0,5.55
156,156,Hermanus,ZA,1589995448,19.23,-34.42,71.11,38,0,3.60
162,162,Santander,ES,1589995448,-3.80,43.46,71.60,64,0,5.82
165,165,Mossamedes,AO,1589995448,12.15,-15.20,77.67,61,0,9.66
208,208,Saldanha,ZA,1589995451,17.94,-33.01,71.60,42,0,2.24
244,244,Arrondissement de Lannion,FR,1589995453,-3.42,48.75,78.80,56,0,5.82
542,542,Tiznit Province,MA,1589995474,-9.50,29.58,78.01,42,0,8.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
hotel_df = narrowed_city_df[["city name", "country", "lat", "lon"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,city name,country,lat,lon,Hotel Name
41,Tigzirt,DZ,36.89,4.12,
105,Malanje,AO,-9.54,16.34,
137,Cidreira,BR,-30.18,-50.21,
156,Hermanus,ZA,-34.42,19.23,
162,Santander,ES,43.46,-3.80,
165,Mossamedes,AO,-15.20,12.15,
208,Saldanha,ZA,-33.01,17.94,
244,Arrondissement de Lannion,FR,48.75,-3.42,
542,Tiznit Province,MA,29.58,-9.50,


In [43]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["lat"]
    lng = row["lon"]
    
    params["location"] = f"{lat},{lng}"    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)    
    name_address = name_address.json()    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,city name,country,lat,lon,Hotel Name
41,Tigzirt,DZ,36.89,4.12,
105,Malanje,AO,-9.54,16.34,
137,Cidreira,BR,-30.18,-50.21,
156,Hermanus,ZA,-34.42,19.23,
162,Santander,ES,43.46,-3.80,
165,Mossamedes,AO,-15.20,12.15,
208,Saldanha,ZA,-33.01,17.94,
244,Arrondissement de Lannion,FR,48.75,-3.42,
542,Tiznit Province,MA,29.58,-9.50,


In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{city name}</dd>
<dt>country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [50]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [51]:
Figure(layout=FigureLayout(height='420px'))

NameError: name 'Figure' is not defined